In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load data from Excel file
data = pd.read_excel('dataset/dataset.xlsx')

# Parse timestamps and set as index
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.set_index('Timestamp', inplace=True)



# Select relevant features
features = ['PQM08 Real Power Mean (kW)', 
            'RmTmp - DH01_ROW01_LA_TTHT01', 
            'RmTmp - DH01_ROW01_LA_TTHT02', 
            'RmTmp - DH01_ROW01_LA_TTHT03', 
            'RmTmp - DH01_ROW01_LA_TTHT04', 
            'RmRhTL - DH01_ROW01_LA_TTHT01', 
            'RmRhTL - DH01_ROW01_LA_TTHT02']

data = data[features]

# preprocess data to remove NaN values
data = data.dropna()

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)


print(data.isnull().sum())  # Untuk mengecek jumlah nilai NaN
print(np.any(np.isnan(scaled_data)))  # Untuk mengecek apakah ada nilai NaN setelah normalisasi
print(np.any(np.isinf(scaled_data)))  # Untuk mengecek apakah ada nilai infinity setelah 


# Print first 5 rows of scaled data
print(scaled_data[:5])

# Tentukan panjang data latih (misalnya 80% dari total data)
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size

# Bagi data menjadi data latih dan data uji
train_data = scaled_data[0:train_size, :]
test_data = scaled_data[train_size:len(scaled_data), :]

# Langkah 2: Memformat Data untuk LSTM
# LSTM memerlukan data yang diformat dalam bentuk tiga dimensi: (jumlah sampel, jumlah time steps, jumlah fitur).
def create_dataset(dataset, time_step=1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), :]
        data_X.append(a)
        data_Y.append(dataset[i + time_step, 1])  # Memilih kolom suhu sebagai target
    return np.array(data_X), np.array(data_Y)

# Menyiapkan data latih untuk model
time_step = 10  # Jumlah time steps
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape input untuk LSTM [jumlah sampel, time steps, jumlah fitur]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(features))
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], len(features))

# Inisialisasi model LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], len(features))))
model.add(LSTM(units=50))
model.add(Dense(1))  # Output layer

# Kompilasi model
model.compile(optimizer='adam', loss='mean_squared_error')

# Melatih model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Evaluasi model
train_loss = model.evaluate(X_train, y_train, verbose=0)
print(f'Training loss: {train_loss}')

test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {test_loss}')




PQM08 Real Power Mean (kW)       0
RmTmp - DH01_ROW01_LA_TTHT01     0
RmTmp - DH01_ROW01_LA_TTHT02     0
RmTmp - DH01_ROW01_LA_TTHT03     0
RmTmp - DH01_ROW01_LA_TTHT04     0
RmRhTL - DH01_ROW01_LA_TTHT01    0
RmRhTL - DH01_ROW01_LA_TTHT02    0
dtype: int64
False
False


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load data from Excel file
data = pd.read_excel('dataset/dataset.xlsx')

# Parse timestamps and set as index
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.set_index('Timestamp', inplace=True)

# Select relevant features
features = ['PQM08 Real Power Mean (kW)', 
            'RmTmp - DH01_ROW01_LA_TTHT01', 
            'RmTmp - DH01_ROW01_LA_TTHT02', 
            'RmTmp - DH01_ROW01_LA_TTHT03', 
            'RmTmp - DH01_ROW01_LA_TTHT04', 
            'RmRhTL - DH01_ROW01_LA_TTHT01', 
            'RmRhTL - DH01_ROW01_LA_TTHT02']

data = data[features]

# Preprocess data to remove NaN values
data = data.dropna()

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Check for NaN and infinity values
print(data.isnull().sum())  # Check for NaN values
print(np.any(np.isnan(scaled_data)))  # Check for NaN values after scaling
print(np.any(np.isinf(scaled_data)))  # Check for infinity values after scaling

# Print first 5 rows of scaled data
print(scaled_data[:5])

# Determine train data length (e.g., 80% of total data)
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size

# Split data into train and test sets
train_data = scaled_data[0:train_size, :]
test_data = scaled_data[train_size:len(scaled_data), :]

# Create dataset for LSTM
def create_dataset(dataset, time_step=1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - time_step):
        a = dataset[i:(i + time_step), :]
        data_X.append(a)
        data_Y.append(dataset[i + time_step, 1])  # Select temperature column as target
    return np.array(data_X), np.array(data_Y)

# Prepare training data for model
time_step = 10  # Number of time steps
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape input for LSTM [number of samples, time steps, number of features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], len(features))
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], len(features))

# Initialize LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], len(features))))
model.add(LSTM(units=50))
model.add(Dense(1))  # Output layer

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Evaluate model
train_loss = model.evaluate(X_train, y_train, verbose=0)
print(f'Training loss: {train_loss}')

test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {test_loss}')

# Hasil Model akan digunakan untuk melakukan prediksi early warning peningkatan suhu pada sebuah server


PQM08 Real Power Mean (kW)       0
RmTmp - DH01_ROW01_LA_TTHT01     0
RmTmp - DH01_ROW01_LA_TTHT02     0
RmTmp - DH01_ROW01_LA_TTHT03     0
RmTmp - DH01_ROW01_LA_TTHT04     0
RmRhTL - DH01_ROW01_LA_TTHT01    0
RmRhTL - DH01_ROW01_LA_TTHT02    0
dtype: int64
False
False
[[0.34869488 0.14371257 0.08985025 0.05882353 0.34660422 0.27756131
  0.29310734]
 [0.35183535 0.14371257 0.08153078 0.05294118 0.34660422 0.27549851
  0.2980791 ]
 [0.34450742 0.13972056 0.08485857 0.0745098  0.34660422 0.27045611
  0.28497175]
 [0.34964632 0.16167665 0.08985025 0.0745098  0.34660422 0.2622049
  0.28610169]
 [0.36538433 0.14371257 0.08485857 0.06862745 0.34192037 0.26953931
  0.29220339]]
Epoch 1/100
732/732 [==============================] - 18s 15ms/step - loss: 4.3872e-04
Epoch 2/100
732/732 [==============================] - 11s 15ms/step - loss: 1.9559e-04
Epoch 3/100
732/732 [==============================] - 11s 15ms/step - loss: 1.8543e-04
Epoch 4/100
732/732 [==============================] - 1

In [3]:
!pip install matplotlib

     ------------------------------------ 138.5/138.5 kB 822.2 kB/s eta 0:00:00
   ---------------------------------------- 7.2/7.2 MB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 965.4/965.4 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 55.8/55.8 kB 484.6 kB/s eta 0:00:00
   ---------------------------------------- 2.5/2.5 MB 1.4 MB/s eta 0:00:00
   -------------------------------------- 103.2/103.2 kB 989.4 kB/s eta 0:00:00


In [2]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install tensorflow
!pip install openpyxl
!pip install flask

   ---------------------------------------- 10.0/10.0 MB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 14.0/14.0 MB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 505.5/505.5 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 7.1/7.1 MB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 302.2/302.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 34.1/34.1 MB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 266.3/266.3 MB 1.4 MB/s eta 0:00:00
   -------------------------------------- 133.7/133.7 kB 875.6 kB/s eta 0:00:00
   ---------------------------------------- 57.5/57.5 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 4.5/4.5 MB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 2.6/2.6 MB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 1.7/1.7 MB 893.5 kB/s eta 0:00:00
   ---------------------------------------- 26.4/26.4 MB 2.1 M